# Processamento de Linguagem Natural - Minicurso do SBBD 2022

# Tradução automática

Esse código foi desenvolvido para o minicurso de PLN no SBBD 2022.

Autoras: Helena Caseli, Cláudia Freitas e Roberta Viola

https://sites.google.com/view/brasileiras-pln/

Fontes:
* Curso de Linguística Computacional da UFMG - Prof. Thiago Castro Ferreira https://www.youtube.com/playlist?list=PLLrlHSmC0Mw73a1t73DEjgGMPyu8QssWT => Este código é baseado na aula 7.8
* https://huggingface.co/
* https://pytorch.org/
* https://www.nltk.org/
* https://www.nltk.org/howto/portuguese_en.html

Esse código:
* Utiliza um modelo neural pré-treinado, baseado na arquitetura Transformer, com fine-tuning para geração de texto.
* Geração de um texto em português a partir de um texto em inglês.

Dataset/corpus:
* TED Corpus

**IMPORTANTE:** Setar a GPU do Colab.

## Instalando as dependências

In [1]:
!pip3 install sentencepiece
!pip3 install transformers
!pip3 install translate-toolkit # Com funcionalidades para manipular textos paralelos
!pip3 install sacremoses

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 5.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 5.1 MB/s 
     |████████████████████████████████| 596 kB 67.1 MB/s 
     |████████████████████████████████| 101 kB 13.2 MB/s 
     |████████████████████████████████| 6.6 MB 46.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 5.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 880 kB 5.0 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0

## Baixando o corpus

Vamos usar o *corpus* de legendas do TED Talks Open Translation Project, com aproximadamente  50000 exemplos de treinamento, 1100 de validação e 2000 de teste.

In [2]:
!wget https://object.pouta.csc.fi/OPUS-TED2020/v1/tmx/en-pt_br.tmx.gz
!gunzip en-pt_br.tmx.gz

--2022-08-17 13:19:08--  https://object.pouta.csc.fi/OPUS-TED2020/v1/tmx/en-pt_br.tmx.gz
Resolving object.pouta.csc.fi (object.pouta.csc.fi)... 86.50.254.18, 86.50.254.19
Connecting to object.pouta.csc.fi (object.pouta.csc.fi)|86.50.254.18|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 32862474 (31M) [application/gzip]
Saving to: ‘en-pt_br.tmx.gz’

en-pt_br.tmx.gz     100%[===================>]  31.34M  15.0MB/s    in 2.1s    

2022-08-17 13:19:11 (15.0 MB/s) - ‘en-pt_br.tmx.gz’ saved [32862474/32862474]



## Carregando o corpus

Lendo e separando em treino e teste. Como o *corpus* original é en-pt e neste código vamos usá-lo no sentido inverso (pt-en), então precisamos apenas tomar cuidado para carregar corretamente quem é fonte (source, src) e quem é alvo (target, trg) na tradução.

Como o modelo pré-treinado que vamos usar foi especificado para várias línguas romanas, é preciso inserir um prefixo indicando a língua que estamos ajustando, no caso: pt_br.

In [3]:
from translate.storage.tmx import tmxfile
from random import shuffle

# lendo o corpus
with open("en-pt_br.tmx", 'rb') as fin:
  f = tmxfile(fin, 'en', 'pt')

prefixo = '>>pt_br<<'
# formatando as traduções corretamente 
#data = [{ 'src': prefixo + ' ' + w.source, 'trg': w.target } for w in f.unit_iter()]
data = [{ 'src': prefixo + ' ' + w.target, 'trg': w.source } for w in f.unit_iter()]

# embaralhando os pares
shuffle(data)
# separando em conjuntos de treino e teste
size = int(len(data) * 0.2)
trainset = data[size:][:10000]
testset = data[:size][:1000]

In [4]:
trainset[15:20]

[{'src': '>>pt_br<< É surpreendente saber que oito palestrantes ontem mencionaram esses termos em suas falas. ',
  'trg': 'You may be surprised to know that eight speakers yesterday actually mentioned these terms in their talks. '},
 {'src': '>>pt_br<< Então, pensei: "Vamos ir além. Vou criar três logotipos, todos com base nessa ideia. ',
  'trg': "So I thought, let's overdeliver. "},
 {'src': '>>pt_br<< Mas havia um problema. ',
  'trg': 'But there was a catch. '},
 {'src': '>>pt_br<< E com essa foto de um belo tubarão-tigre fêmea de 4,5; 4 metros provavelmente, eu acho, acho que alcancei esse objetivo, ela estava nadando com esses pequenos xereletes azuis no nariz e meu flash criou uma sombra em seu rosto. ',
  'trg': 'And with this photograph of a beautiful 15-feet, probably 14-feet, I guess, female tiger shark, I sort of think I got to that goal, where she was swimming with these little barjacks off her nose, and my strobe created a shadow on her face. '},
 {'src': '>>pt_br<< Eu nã

Setando os parâmetros da rede neural.

In [5]:
nepochs = 3
batch_size = 16
batch_status = 32
learning_rate = 1e-5 # usar uma bem baixa para o caso dos modelos pré-treinados
early_stop = 2
write_path = 'model.pt' # caminho para salvar o melhor modelo

Separando os dados de treinamento e teste em lotes.

In [6]:
from torch.utils.data import DataLoader, Dataset

traindata = DataLoader(trainset, batch_size=batch_size, shuffle=True)
devdata = DataLoader(testset, batch_size=batch_size, shuffle=True)

Definindo o método de avaliação.

In [7]:
from nltk.translate.bleu_score import corpus_bleu
def evaluate(tokenizer, model, devdata, batch_size, batch_status, device):
    model.eval()
    y_real = []
    y_pred = []
    for batch_idx, inp in enumerate(devdata):
        y_real.extend(inp['trg'])
        # tokenizando
        model_inputs = tokenizer(inp['src'], truncation=True, padding=True, max_length=128, return_tensors="pt").to(device)
        # traduzindo
        generated_ids = model.generate(**model_inputs, num_beams=1) # gera token por token
        # pos-processando a traducao, decodificando no texto de saida
        output = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
        y_pred.extend(output)
    
        # imprimindo o andamento
        if (batch_idx+1) % batch_status == 0:
            print('Avaliacao: [{}/{} ({:.0f}%)]'.format(batch_idx+1, \
                len(devdata), 100. * batch_idx / len(devdata)))

    # avaliando com base no BLEU
    hyps, refs = [], []
    for i, snt_pred in enumerate(y_pred):
        hyps.append(nltk.word_tokenize(snt_pred))
        refs.append([nltk.word_tokenize(y_real[i])])
    bleu = corpus_bleu(refs, hyps)

    return bleu

## Treinamento

**IMPORTANTE:** Setar a GPU do Colab.

Definindo o método de treinamento.

In [8]:
def train(tokenizer, model, traindata, devdata, optimizer, num_epochs, batch_size, batch_status, device, early_stop=5, write_path='model.pt'):
  max_bleu = evaluate(tokenizer, model, devdata, batch_size, batch_status, device)
  print('BLEU inicial:', max_bleu)
  model.train()
  repeat = 0
  for epoch in range(num_epochs):
    losses = []
    batch_src, batch_trg = [], []

    for batch_idx, inp in enumerate(traindata):
        # inicializando
        optimizer.zero_grad()

        # tokenizando
        model_inputs = tokenizer(inp['src'], truncation=True, padding=True, max_length=128, return_tensors="pt").to(device)
        with tokenizer.as_target_tokenizer():
          labels = tokenizer(inp['trg'], truncation=True, padding=True, max_length=128, return_tensors="pt").input_ids.to(device)
        # traduzindo
        output = model(**model_inputs, labels=labels) # forward pass

        # calculando a loss (erro)
        loss = output.loss
        losses.append(float(loss))

        # Backpropagation com base no erro
        loss.backward()
        optimizer.step()

        batch_src, batch_trg = [], []

        # Imprimindo o andamento
        if (batch_idx+1) % batch_status == 0:
            print('Epoca de treinamento: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tTotal Loss: {:.6f}'.format(
            epoch, batch_idx+1, len(traindata), 100. * batch_idx / len(traindata), float(loss), round(sum(losses) / len(losses), 5)))
    
    bleu = evaluate(tokenizer, model, devdata, batch_size, batch_status, device)
    print('BLEU:', bleu)
    if bleu > max_bleu:
        max_bleu = bleu
        repeat = 0

        print('Salvando o melhor modelo ...')
        torch.save(model, write_path)
    else:
        repeat += 1

    if repeat == early_stop:
        break

Inicializando o modelo.

In [9]:
import nltk
nltk.download('punkt')
import torch
from torch import optim
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-ROMANCE").to(device) # esse eh o modelo pre-treinado que vamos usar
#tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-ROMANCE") # o pt_br inserido anteriormente indica qual eh a lingua alvo
model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-ROMANCE-en").to(device) # esse eh o modelo pre-treinado que vamos usar
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-ROMANCE-en") # o pt_br inserido anteriormente indica qual eh a lingua fonte


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Treinando.

In [10]:
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
train(tokenizer, model, traindata, devdata, optimizer, nepochs, batch_size, batch_status, device, early_stop, write_path)

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1207: UserWarning: Neither `max_length` nor `max_new_tokens` have been set, `max_length` will default to 512 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,
/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1207: UserWarning: Neither `max_length` nor `max_new_tokens` have been set, `max_length` will default to 512 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


Avaliacao: [32/63 (49%)]
BLEU inicial: 0.42932714976529174
Epoca de treinamento: 0 [32/625 (5%)]	Loss: 0.826099	Total Loss: 1.622830
Epoca de treinamento: 0 [64/625 (10%)]	Loss: 0.471779	Total Loss: 1.122830
Epoca de treinamento: 0 [96/625 (15%)]	Loss: 0.569928	Total Loss: 0.938030
Epoca de treinamento: 0 [128/625 (20%)]	Loss: 0.637050	Total Loss: 0.827700
Epoca de treinamento: 0 [160/625 (25%)]	Loss: 0.637415	Total Loss: 0.769160
Epoca de treinamento: 0 [192/625 (31%)]	Loss: 0.484038	Total Loss: 0.728740
Epoca de treinamento: 0 [224/625 (36%)]	Loss: 0.590291	Total Loss: 0.694780
Epoca de treinamento: 0 [256/625 (41%)]	Loss: 0.534876	Total Loss: 0.673420
Epoca de treinamento: 0 [288/625 (46%)]	Loss: 0.478043	Total Loss: 0.653570
Epoca de treinamento: 0 [320/625 (51%)]	Loss: 0.658871	Total Loss: 0.638340
Epoca de treinamento: 0 [352/625 (56%)]	Loss: 0.485488	Total Loss: 0.626540
Epoca de treinamento: 0 [384/625 (61%)]	Loss: 0.400428	Total Loss: 0.614470
Epoca de treinamento: 0 [416/625 

## Predição

Testando o modelo para algumas sentenças, entre elas a sentença de exemplo do Capítulo que acompanha o minicurso.

In [12]:
# sentenças a serem traduzidas

# en -> pt
#batch_input_str = ((">>pt_br<< We can do better, America can do better, and help is on the way."), 
#                   (">>pt_br<< The boy went to school by bus."), 
#                   (">>pt_br<< Today is a happy day because we are in Búzios!"))

# pt -> en
batch_input_str = (("Hoje é um grande dia!"), 
                   ("O menino foi para a escola de ônibus."), 
                   ("O menino foi de ônibus para a escola."), 
                   ("Com estas palavras, André Coruja, além de quebrar o gelo que havia esfriado o clima, devolveu ao recinto a eloquência necessária para que a sessão continuasse."))

# tokenizando as sentenças
encoded = tokenizer(batch_input_str, return_tensors='pt', padding=True).to(device)
# traduzindo
translated = model.generate(**encoded)
# preparando a saída
tokenizer.batch_decode(translated, skip_special_tokens=True)

["Today's a big day!",
 'The boy went to bus school.',
 'The boy went by bus to school.',
 'With these words, André Oruja, in addition to breaking the ice that had cooled the climate, returned to the compound the eloquence necessary for the session to continue.']

Fim deste exemplo.